<a href="https://colab.research.google.com/github/AnikaZN/DS-Unit-2-Linear-Models/blob/master/Linear_Models_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Data

In [45]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/nyc/nyc-rent-2016.csv')
df.head()

bathrooms  bedrooms  ... common_outdoor_space wheelchair_access
0        1.5         3  ...                    0                 0
1        1.0         2  ...                    0                 0
2        1.0         1  ...                    0                 0
3        1.0         1  ...                    0                 0
4        1.0         4  ...                    0                 0

[5 rows x 34 columns]

In [0]:
df['outdoor'] = (df['roof_deck'] + df['outdoor_space'] + df['balcony'] + df['swimming_pool'] +
                 df['terrace'] + df['garden_patio'] + df['common_outdoor_space'])

# Train/Test Split

In [76]:
df['created'] = pd.to_datetime(df['created'], infer_datetime_format=True)
df['created'].head()

0   2016-06-24 07:54:24
1   2016-06-12 12:19:27
2   2016-04-17 03:26:41
3   2016-04-18 02:22:02
4   2016-04-28 01:32:41
Name: created, dtype: datetime64[ns]

In [0]:
df['month'] = df['created'].dt.month

train = df[df['month'] < 6] #April and May
test = df[df['month'] == 6] #June

In [78]:
train.shape, test.shape

((31515, 36), (16785, 36))

In [79]:
train['price'].mean()

3432.7534190068222

# Baseline

In [0]:
from sklearn.metrics import mean_absolute_error as mae

In [81]:
y_test = test['price']
y_pred = [train['price'].mean()] * len(y_test)

print(len(y_test), len(y_pred))
print('Mean Absolute Error:', mae(y_test, y_pred))

16785 16785
Mean Absolute Error: 1052.6798017842948


# Features

In [82]:
df.columns

Index(['bathrooms', 'bedrooms', 'created', 'description', 'display_address',
       'latitude', 'longitude', 'price', 'street_address', 'interest_level',
       'elevator', 'cats_allowed', 'hardwood_floors', 'dogs_allowed',
       'doorman', 'dishwasher', 'no_fee', 'laundry_in_building',
       'fitness_center', 'pre-war', 'laundry_in_unit', 'roof_deck',
       'outdoor_space', 'dining_room', 'high_speed_internet', 'balcony',
       'swimming_pool', 'new_construction', 'exclusive', 'terrace', 'loft',
       'garden_patio', 'common_outdoor_space', 'wheelchair_access', 'outdoor',
       'month'],
      dtype='object')

In [0]:
from sklearn.linear_model import LinearRegression as linreg

In [84]:
model = linreg()
model

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
features = ['bedrooms',
           'bathrooms',
           'longitude', #approx 50 points
           'doorman', #approx 12 points
           'laundry_in_unit', #approx 4 points
           'interest_level', #approx 30 points
           'elevator', #approx 3 points
           'outdoor']

target = 'price'

x_train = train[features]
y_train = train[target]

x_test = test[features]
y_test = test[target]

In [86]:
x_train.shape, y_train.shape

((31515, 8), (31515,))

# One-hot Encoding

In [87]:
! pip install category_encoders

In [0]:
import category_encoders as ce

encoder = ce.OneHotEncoder(use_cat_names=True)

x_train = encoder.fit_transform(x_train)
x_test = encoder.transform(x_test)

In [97]:
model.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [98]:
model.coef_

array([ 556.51019224,  527.70986048, -362.0728824 ,  190.23700025,
        123.97820498, -110.48155283,  133.73220208,  -75.98758244,
         83.83276837,  -47.55529396])

In [99]:
model.intercept_

3432.7534190069305

# Scale

In [0]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [101]:
x_train

array([[-0.45807512, -0.42056347, -0.99016991, ..., -0.55395212,
        -1.0537678 , -0.51025394],
       [-0.45807512, -0.42056347,  0.16078122, ..., -0.55395212,
        -1.0537678 , -0.51025394],
       [ 2.28697859, -0.42056347,  0.7818223 , ..., -0.55395212,
        -1.0537678 , -0.51025394],
       ...,
       [-0.45807512, -0.42056347, -0.59189356, ..., -0.55395212,
         0.94897567, -0.51025394],
       [-1.37309302, -0.42056347, -1.27031344, ..., -0.55395212,
         0.94897567, -0.51025394],
       [ 0.45694278, -0.42056347,  1.86526896, ..., -0.55395212,
        -1.0537678 , -0.51025394]])

# MAE

In [102]:
y_pred = model.predict(x_test)
y_pred

array([3638.06437429, 3829.77573039, 2436.45212414, ..., 2605.79303248,
       3708.31912008, 3458.90188635])

In [104]:
mae(y_test, y_pred)

600.9909685378111